This works off of
https://github.com/pc494/pyxem/commit/48b6c6b8bb1445bcb1913bdc92b8b24a3fb1280a

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyxem as pxm
import pymatgen as pmg
from matplotlib import pyplot as plt
import numpy as np
from pyxem.utils.pyprismatic_io_utils import *
from matplotlib import pyplot as plt
import pyprismatic as pr
import os
%matplotlib tk

First we set up the structure

In [3]:
E = pmg.Element("Pt")
cubic_lattice = pmg.Lattice.cubic(5)
Cubic_P = pmg.Structure.from_spacegroup(195,cubic_lattice, [E], [[0, 0, 0]])
ediff = pxm.ElectronDiffractionCalculator(300., 1/(40*2))

In [16]:
def load_default_params():
    meta_params = {}
    ### Computational
    meta_params['save4DOutput'] = True
    meta_params['save3DOutput'] = False
    meta_params['sliceThickness'] = 1.0
    meta_params['algorithm']='multislice'
    ### STEM
    delta = 1e-10 #only get one STEM pattern
    meta_params['scanWindowXMin'] = 0.5-delta
    meta_params['scanWindowXMax'] = 0.5+delta
    meta_params['scanWindowYMin'] = 0.5-delta
    meta_params['scanWindowYMax'] = 0.5+delta
    ### Experimental
    #meta_params['probeSemiangle'] = 0.01
    #meta_params['alphaBeamMax'] = 0.015
    meta_params['E0']=300e3
    ### Sample
    meta_params['tileX']=10
    meta_params['tileY']=10
    meta_params['tileZ']=50 #1000 kills you
    meta_params['realspacePixelSizeX']=0.025
    meta_params['realspacePixelSizeY']=0.025 #default is 0.1, smaller number more pixels
    return meta_params

In [5]:
def prismatic_experiments(variable_names,list_of_values):
    """
    variables_names: list
    list_of_values : list of values, can't be used to vary two params, eg) X and Y must stay the same
    """
    ### get defaults in
    import os
    diff_list = []
    meta_params = load_default_params()
    for variable in list_of_values:
        try:
            os.remove('PP_input.XYZ')
        except FileNotFoundError:
            pass
        for i in np.arange(0,len(variable_names)):
            meta_params[variable_names[i]] = variable
        diff_dat_m = ediff.calculate_ed_data_dynamic(Cubic_P,delete_mode=False,prismatic_kwargs=meta_params)
        diff_list.append(diff_dat_m)
        print('One more simulation completed')
    return diff_list

In [22]:
energy_list = [80e3,160e3,300e3]
in_plane_list = [5,10,20,30,50]
out_plane_list = [40]

#energy = prismatic_experiments(['E0'],energy_list)
#in_plane = prismatic_experiments(['tileX','tileY'],in_plane_list)
#out_plane = prismatic_experiments(['tileZ'],out_plane_list)

/home/pc494/anaconda3/lib/python3.6/site-packages/pyxem-0.5-py3.6-linux-x86_64.egg/pyxem/diffraction_generator.py:165: UserWarning: This functionality is a work in progress 
  warnings.warn("This functionality is a work in progress ")


One more simulation completed


In [64]:
for i in np.arange(0,len(energy_list)):
    plt.figure()
    plt.imshow(np.log(energy[i][0,0]))
    plt.title('E0 = ' + str(energy_list[i]/1000) + ' keV')
    plt.show()

/home/pc494/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
for i in np.arange(0,len(in_plane_list)):
    plt.figure()
    plt.imshow(in_plane[i][0,0])
    plt.title('in_plane = ' + str(in_plane_list[i]))
    plt.show()

In [23]:
for i in np.arange(0,len(out_plane_list)):
    plt.figure()
    plt.imshow(np.log(out_plane[i][0,0]))
    plt.title('out_plane = ' + str(out_plane_list[i]))
    plt.show()
    #dp = ElectronDiffraction((out_plane[i][0,0]))
    #dp.set_calibration(1/2480)
    #dp.plot()

/home/pc494/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
np.sqrt(np.sum(out_plane[2][0,0]>0))

504.0

In [82]:
from pyxem import ElectronDiffraction

dp = ElectronDiffraction((energy[0][0,0]))
dp.set_calibration(1/2480)
dp.plot()

In [19]:
plt.close('all')

Below is the basis for the first unit test. That making a supercell in pymatgen is sa good as doing so in pyprismatic.

In [ ]:
os.remove('PP_input.XYZ')
diff_dat_them = ediff.calculate_ed_data_dynamic(Cubic_P,delete_mode=False,prismatic_kwargs=meta_params)

Cubic_P.make_supercell([30,30,100])
meta_params['tileX']=1
meta_params['tileY']=1
meta_params['tileZ']=1

os.remove('PP_input.XYZ')
diff_dat_us = ediff.calculate_ed_data_dynamic(Cubic_P,delete_mode=False,prismatic_kwargs=meta_params)

In [69]:
plt.figure()
plt.imshow(diff_dat_them[0,0])
plt.plot()
plt.figure()
plt.imshow(diff_dat_us[0,0])

(diff_dat_them == diff_dat_us).all()

True